In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlops-zoomcamp-data/yellow_tripdata_2022-02.parquet
/kaggle/input/mlops-zoomcamp-data/yellow_tripdata_2022-01.parquet


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#READ IN DATA
jan_df = pd.read_parquet('/kaggle/input/mlops-zoomcamp-data/yellow_tripdata_2022-01.parquet')
feb_df = pd.read_parquet('/kaggle/input/mlops-zoomcamp-data/yellow_tripdata_2022-02.parquet')

In [3]:
#Q1
jan_df.shape

(2463931, 19)

In [4]:
jan_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [6]:
jan_df['tpep_dropoff'] = pd.to_datetime(jan_df['tpep_dropoff_datetime'])
feb_df['tpep_dropoff'] = pd.to_datetime(feb_df['tpep_dropoff_datetime'])
jan_df['tpep_pickup'] = pd.to_datetime(jan_df['tpep_pickup_datetime'])
feb_df['tpep_pickup'] = pd.to_datetime(feb_df['tpep_pickup_datetime'])

In [7]:
#Q2
#Compute duration
jan_df['duration'] = jan_df['tpep_dropoff'] - jan_df['tpep_pickup']
feb_df['duration'] = feb_df['tpep_dropoff'] - feb_df['tpep_pickup']
# Convert to minutes and seconds
jan_df['duration_min'] = jan_df.duration.apply(lambda td: td.total_seconds() / 60)
feb_df['duration_min'] = feb_df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
jan_df['duration_min'].std()

46.44530513776802

In [9]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,tpep_dropoff,tpep_pickup,duration,duration_min
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,...,3.65,0.0,0.3,21.95,2.5,0.0,2022-01-01 00:53:29,2022-01-01 00:35:40,0 days 00:17:49,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,...,4.00,0.0,0.3,13.30,0.0,0.0,2022-01-01 00:42:07,2022-01-01 00:33:43,0 days 00:08:24,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,...,1.76,0.0,0.3,10.56,0.0,0.0,2022-01-01 01:02:19,2022-01-01 00:53:21,0 days 00:08:58,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,...,0.00,0.0,0.3,11.80,2.5,0.0,2022-01-01 00:35:23,2022-01-01 00:25:21,0 days 00:10:02,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,...,3.00,0.0,0.3,30.30,2.5,0.0,2022-01-01 01:14:20,2022-01-01 00:36:48,0 days 00:37:32,37.533333


In [10]:
#Q3
((jan_df['duration_min'] >= 1) & (jan_df['duration_min'] <= 60)).mean()

0.9827547930522406

In [11]:
#Outlier Removal
jan_df = jan_df[(jan_df['duration_min'] >= 1) & (jan_df['duration_min'] <= 60)]
feb_df = feb_df[(feb_df['duration_min'] >= 1) & (feb_df['duration_min'] <= 60)]

In [12]:
#Q4
categorical = ['PULocationID', 'DOLocationID']
jan_df[categorical] = jan_df[categorical].astype(str)
train_dicts = jan_df[categorical].to_dict(orient='records')

In [13]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [14]:
X_train.shape

(2421440, 515)

In [15]:
#Q5
target = 'duration_min'
y_train = jan_df[target].values
y_val = feb_df[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)

In [18]:
mean_squared_error(y_train, y_pred, squared=False)

6.986190135957529

In [19]:
#Q6
#Create validation set
feb_df[categorical] = feb_df[categorical].astype(str)
valid_dicts = feb_df[categorical].to_dict(orient='records')
X_val = dv.transform(valid_dicts)

In [20]:
X_val.shape

(2918187, 515)

In [21]:
y_pred = lr.predict(X_val)

In [22]:
mean_squared_error(y_val, y_pred, squared=False)

7.786389537638445